In [14]:
import warnings
warnings.filterwarnings('ignore')

##### Validación cruzada

In [15]:
import pandas as pd
from joblib import load
import numpy as np

from sklearn.model_selection import cross_val_score

##### Cargar modelos entrenados

In [16]:
models = {
    "KNN": load("results/knn_model.joblib"),
    "DT": load("results/dt_model.joblib"),
    "SVM": load("results/svm_model.joblib"),
    "ADA": load("results/ada_model.joblib"),
    "GBC": load("results/gbc_model.joblib"),
    "RF": load("results/rf_model.joblib")
}

##### Leer datos y respuestas

In [17]:
train_data = pd.read_csv("process_dataset/train_data_scaled_robust.csv")
val_data = pd.read_csv("process_dataset/val_data.csv")
test_data = pd.read_csv("process_dataset/test_data.csv")

y_train = np.load("process_dataset/y_train.npy")
y_val = np.load("process_dataset/y_val.npy")
y_test = np.load("process_dataset/y_test.npy")

#### Ejecutar validación cruzada y calcular métricas de desempeño

In [18]:
results = []

for model_name, model in models.items():
    metrics = {
        "Model": model_name,
        "Accuracy": cross_val_score(model, train_data, y_train, cv=10, scoring="accuracy").mean(),
        "Recall": cross_val_score(model, train_data, y_train, cv=10, scoring="recall").mean(),
        "Precision": cross_val_score(model, train_data, y_train, cv=10, scoring="precision").mean(),
        "F1": cross_val_score(model, train_data, y_train, cv=10, scoring="f1").mean()
    }
    results.append(metrics)

df_results = pd.DataFrame(results)

In [19]:
df_results

,Model,Accuracy,Recall,Precision,F1
0,KNN,0.492920,0.467224,0.484266,0.475002
1,DT,0.508407,0.507197,0.501650,0.506589
2,SVM,0.499336,0.466776,0.490756,0.478199
3,ADA,0.503540,0.460875,0.495478,0.477284
4,GBC,0.505088,0.436674,0.496511,0.463035
5,RF,0.501549,0.419586,0.487328,0.459305
